In [ ]:
# @title 🚀 1. PREPARAR AMBIENTE
import os, subprocess, time, random
print("🧹 Limpando sessões antigas...")
!pkill -9 -f engine.py
!pkill -9 cloudflared
!rm -f engine.py*

print("📦 Instalando dependências (Aguarde...)")
!pip install -q diffusers transformers accelerate flask flask-cors

print("📥 Baixando engine.py...")
v = random.randint(1, 99999)
!wget -q "https://raw.githubusercontent.com/Miraplay2025/Gerador-de-imagem/main/engine.py?v={v}" -O engine.py
print("✅ Tudo pronto. Execute a célula abaixo para ligar o servidor.")

In [ ]:
# @title 🔗 2. INICIAR MOTOR E MANTER ONLINE
import subprocess, re, time, sys

print("⚙️ Iniciando servidor Flask...")
subprocess.Popen(["python3", "engine.py"])

print("⏳ Aguardando 15 segundos para a IA carregar na GPU...")
time.sleep(15)

print("🌐 Ativando túnel Cloudflare...")
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb > /dev/null 2>&1

# Inicia o túnel
p = subprocess.Popen(['cloudflared', 'tunnel', '--url', 'http://127.0.0.1:5000'], 
                     stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

print("🔍 Procurando link público...")
url_found = False
try:
    for line in p.stdout:
        # Imprime o log do cloudflare para você ver o que está acontecendo
        if ".trycloudflare.com" in line and not url_found:
            url = re.search(r"https://[a-zA-Z0-9-]+\.trycloudflare\.com", line)
            if url:
                print("\n" + "="*60)
                print(f"🚀 SISTEMA ONLINE: {url.group(0)}")
                print("="*60)
                print("MANTENHA ESTA ABA ABERTA PARA O GERADOR FUNCIONAR")
                url_found = True
        
        # Isso mantém a célula rodando indefinidamente
        if url_found:
            # Opcional: imprimir pontos para mostrar que está vivo
            time.sleep(1)
except KeyboardInterrupt:
    print("\nEncerrando servidor...")
    p.terminate()